# Introduction

In [13]:
import pandas as pd

exoplanet_data = pd.read_csv('data/2024-03-01-exoplanet_dataset.csv', 
                             header = 13,
                             dtype = {'pl_name' : 'string', 'st_spectype' : 'string'})
print(exoplanet_data)

         pl_name st_spectype  sy_umag  sy_gmag  sy_rmag  sy_imag  sy_zmag
0       11 Com b        <NA>      NaN      NaN      NaN      NaN      NaN
1       11 Com b      G8 III      NaN      NaN      NaN      NaN      NaN
2       11 Com b      G8 III      NaN      NaN      NaN      NaN      NaN
3       11 UMi b        <NA>      NaN      NaN      NaN      NaN      NaN
4       11 UMi b        <NA>      NaN      NaN      NaN      NaN      NaN
...          ...         ...      ...      ...      ...      ...      ...
35843  ups And d        <NA>      NaN      NaN      NaN      NaN      NaN
35844  ups Leo b      G9 III      NaN      NaN      NaN      NaN      NaN
35845   xi Aql b        <NA>      NaN      NaN      NaN      NaN      NaN
35846   xi Aql b        <NA>      NaN      NaN      NaN      NaN      NaN
35847   xi Aql b      K0 III      NaN      NaN      NaN      NaN      NaN

[35848 rows x 7 columns]


This dataset from NASA's Exoplanet Archive include all planets and stars. Therefore we will wrangle the dataset such that it only contain stars with Sloan magnitudes for photometric measurements.

In [22]:
only_stars_data = exoplanet_data.dropna(subset = ['st_spectype',
                                                  'sy_umag',
                                                  'sy_gmag',
                                                  'sy_rmag',
                                                  'sy_imag',
                                                  'sy_zmag'])

only_stars_data['st_spectype'].transform()
print(only_stars_data)

SyntaxError: invalid syntax (2297352337.py, line 8)